# Drive mount

In [1]:
from google.colab import drive

drive.mount('/drive')

Mounted at /drive


# Preprocess

In [6]:
import os
import xml.etree.cElementTree as ET
import cv2
from PIL import Image

prohibitory=[0,1,2,3,4,5,7,8,9,10,15,16]#circular,white ground with red border  
mandatory=[33,34,35,36,37,38,39,40]#circular,blue ground
danger=[11, 18, 19, 20,21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]#triangular,red ground  
other= [6,12,13,14,17,32,41,42]

ANNOTATIONS_DIR_PREFIX = '/drive/My Drive/GTSDB/'

DESTINATION_DIR = "/drive/My Drive/GTSDB/Annotations2"

def create_root(filename, width, height):
  root = ET.Element("annotation")
  ET.SubElement(root, "folder").text = "Images"
  ET.SubElement(root, "filename").text = (filename)
  ET.SubElement(root, "path").text = "Images/{}".format(filename)
  source = ET.SubElement(root, "source")
  ET.SubElement(source, "database").text = "Unknown"
  size = ET.SubElement(root, "size")
  ET.SubElement(size, "width").text = str(width)
  ET.SubElement(size, "height").text = str(height)
  ET.SubElement(size, "depth").text = "3"
  ET.SubElement(root, "segmented").text = "0"
  return root


def create_object_annotation(root, voc):
  for voc_label in voc:
   obj = ET.SubElement(root, "object")
   ET.SubElement(obj, "name").text= voc_label[0]
   ET.SubElement(obj, "pose").text = "Unspecified"
   ET.SubElement(obj, "truncated").text = str(0)
   ET.SubElement(obj, "difficult").text = str(0)
   bbox = ET.SubElement(obj, "bndbox")
   ET.SubElement(bbox, "xmin").text = str(voc_label[1])
   ET.SubElement(bbox, "ymin").text = str(voc_label[2])
   ET.SubElement(bbox, "xmax").text = str(voc_label[3])
   ET.SubElement(bbox, "ymax").text = str(voc_label[4])
  return root


def create_file(filename, width, height, voc):
  root = create_root(filename, width, height)
  root = create_object_annotation(root, voc)
  tree = ET.ElementTree(root)
  tree.write("{}/{}.xml".format(DESTINATION_DIR, filename[:-4]))


def read_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
        for image in os.listdir("/drive/My Drive/GTSDB/Images"):
            voc_labels = []
            img = Image.open("{}/{}".format("/drive/My Drive/GTSDB/Images", image))
            w, h = img.size
            for line in lines:
                voc = []
                line = line.strip()
                data = line.split(';')
                if data[0] == image:
                    finalClass = ''
                    if int(data[5]) in prohibitory:
                      finalClass = 'prohibitory'
                    if int(data[5]) in mandatory:
                      finalClass = 'mandatory'
                    elif int(data[5]) in danger:
                      finalClass = 'danger'
                    elif int(data[5]) in other:
                      finalClass = 'other'
                    if finalClass == '':
                      print('Fuck')
                    voc.append(finalClass)
                    voc.append(data[1])
                    voc.append(data[2])
                    voc.append(data[3])
                    voc.append(data[4])
                    voc_labels.append(voc)
            create_file(image, w, h, voc_labels)
    print("Processing complete for file: {}".format(file_path))

read_file('/drive/My Drive/GTSDB/gt.txt')

Processing complete for file: /drive/My Drive/GTSDB/gt.txt
